In [1]:
!pip install torch
!pip install transformers
!pip install --upgrade pip
!conda install -y -c pytorch faiss-cpu
!pip install sentence_transformers
!pip install sacremoses

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, FlaubertModel, pipeline
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import os

In [3]:
model_name_llm = 'microsoft/Phi-3-mini-128k-instruct'
model_name_embedding = "dangvantuan/sentence-camembert-large"

In [4]:
path_embeddings = '/home/onyxia/work/StatApp-RAG/camemBERT/embeddings_doc2.csv'
path_text_and_embeddings = '/home/onyxia/work/StatApp-RAG/camemBERT/text_and_embeddings_doc2.csv'
path_index = '/home/onyxia/work/StatApp-RAG/camemBERT/faiss_index_doc2'
path_QA = '/home/onyxia/work/StatApp-RAG/Q&A/output_triplets_2.csv'

In [5]:
from huggingface_hub import notebook_login

# Cela ouvrira une invite dans votre notebook pour entrer votre token d'accès
notebook_login()

In [9]:
!pip install sentencepiece --index-url=https://pypi.org/simple
!pip install protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 22.8 MB/s eta 0:00:00


In [10]:
model_embedding = FlaubertModel.from_pretrained(model_name_embedding)
tokenizer_embedding = AutoTokenizer.from_pretrained(model_name_embedding)

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type camembert to instantiate a model of type flaubert. This is not supported for all configurations of models and can yield errors.
Some weights of FlaubertModel were not initialized from the model checkpoint at dangvantuan/sentence-camembert-large and are newly initialized: ['attentions.0.k_lin.bias', 'attentions.0.k_lin.weight', 'attentions.0.out_lin.bias', 'attentions.0.out_lin.weight', 'attentions.0.q_lin.bias', 'attentions.0.q_lin.weight', 'attentions.0.v_lin.bias', 'attentions.0.v_lin.weight', 'attentions.1.k_lin.bias', 'attentions.1.k_lin.weight', 'attentions.1.out_lin.bias', 'attentions.1.out_lin.weight', 'attentions.1.q_lin.bias', 'attentions.1.q_lin.weight', 'attentions.1

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model_llm = AutoModelForCausalLM.from_pretrained(model_name_llm, device_map="cuda", torch_dtype="auto", 
    trust_remote_code=True, )

# Load the tokenizer
tokenizer_llm = AutoTokenizer.from_pretrained(model_name_llm)

model_llm = model_llm.to('cuda')

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Initialisation du pipeline de génération de texte
generator_llm = pipeline('text-generation', model=model_llm, tokenizer=tokenizer_llm, max_length=3000)

generation_args = {
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

def generate_response(question, context):
    # Construction de la requête en utilisant le contexte et la question fournis
    prompt = f"[INST] Tu es un assistant respectueux et utile, réponds toujours de manière précise, assertive et polie en quelques mots en français conversationnel. " \
             f"Contexte : {context} " \
             f"Réponds précisément à la question ci-dessous en 300 caractères avec l'aide du contexte et ne pose pas de question, et réponds en français : " \
             f"{question} [/INST]"

    # Génération de la réponse en utilisant le modèle
    responses = generator_llm(prompt, num_return_sequences=1, **generation_args)
    response = responses[0]['generated_text'] if responses else None
    response = response.replace(prompt, "").strip()
    return response

In [13]:
embeddings = pd.read_csv(path_embeddings).drop('Unnamed: 0', axis=1)
QA = pd.read_csv(path_QA)
text_and_embeddings = pd.read_csv(path_text_and_embeddings)
index = faiss.read_index(path_index)

In [14]:
question = "Quels sont les fonds propres de l'entreprise?"

In [15]:
tokens = tokenizer_embedding.encode(question, add_special_tokens=True, return_tensors="pt")
with torch.no_grad():
    outputs = model_embedding(tokens).last_hidden_state
question_embedding = torch.mean(outputs, dim=1)
D, I = index.search(question_embedding, 3)
context = [text_and_embeddings['text'].iloc[indice] for indice in I[0]]

print(generate_response(question, "\n".join(context)))
print("La réponse est basée sur le context trouvé dans le document 2:", context)

/opt/mamba/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


Les fonds propres de Sogécap ne sont pas spécifiés dans le contexte fourni.
La réponse est basée sur le context trouvé dans le document 2: ['Au sein des risques de marché, Sogécap est sensible aux chocs sur le niveau des taux d’intérêt, des marchés actions et des spreads de crédit. En lien avec l’activité d’assurance vie épargne, le risque de rachats, au sein des risques de souscription vie, est également significatif.Les événements majeurs des derniers exercices, la crise sani-taire puis la guerre en Ukraine qui se poursuit, ont entraîné des répercussions macro-économiques importantes et ont renforcé le niveau d’incertitude. Leurs répercussions sur Sogécap ont principalement porté sur le volume d’activité, mais la société n’a pas connu d’évolution majeure de son profil de risque.', 'Au sein des risques de marché, Sogécap est sensible aux chocs sur le niveau des taux d’intérêt, des marchés actions et des spreads de crédit. En lien avec l’activité d’assurance vie épargne, le risque de r